In [ ]:
import math
from typing import List, Tuple

def sinc_filter_order_new_py(data_in: List[int],
                             osr: int,
                             order_i: int) -> Tuple[List[int], int]:
    """
    Python 版 sinc(CIC) 滤波器实现，对应你的 C 函数 sinc_filter_order_new。

    参数:
        data_in : 输入数据（整型列表）
        osr     : 过采样比 (典型 64, 128, 256, 512；代码也支持 32)
        order_i : 0 表示 sinc2；1 表示 sinc3

    返回:
        (d_out, d_out_len)
        d_out 为输出整数列表，d_out_len 为其长度

    说明:
        - 与 C 版一致，输出长度 = floor(len(data_in)/osr) - (order_i + 1)
        - 丢弃前 (order_i+1) 个抽取点的 comb 输出（sinc2 丢 1 个，sinc3 丢 2 个）
        - 积分器与梳状器之间、以及梳状器级间的差分都按 data_full 位宽取模回绕
        - 重定标因子按你给出的各 OSR 分支实现（32/64/128/256/512）
    """
    if order_i not in (0, 1):
        raise ValueError("order_i 只能为 0(sinc2) 或 1(sinc3)")

    input_len = len(data_in)
    print()
    if input_len == 0:
        return [], 0

    # 输出基础长度（抽 decimate 后的点数）
    len2 = input_len // osr
    d_out_len = len2 - (order_i + 1)
    if d_out_len <= 0:
        return [], 0

    # 2 阶或 3 阶
    order = 2 if order_i == 0 else 3

    # 数据位宽（与你的 C 计算一致）
    data_full = int(math.log2(osr)) * order + 1
    mod = 1 << data_full

    # 预分配
    i1 = [0] * input_len
    i2 = [0] * input_len
    i3 = [0] * input_len
    s  = [0] * len2
    d1 = [0] * len2
    d2 = [0] * len2
    d3 = [0] * len2

    # 积分器
    cnt = 0
    for n in range(input_len):
        if n == 0:
            i1[n] = data_in[0] % mod
            i2[n] = i1[0]
            i3[n] = i2[0]
        else:
            i1[n] = (i1[n-1] + data_in[n]) % mod
            i2[n] = (i2[n-1] + i1[n]) % mod
            i3[n] = (i3[n-1] + i2[n]) % mod

        # 抽取点做 comb（差分）
        if n % osr == osr - 1:
            cnt += 1
            s[cnt - 1] = i3[n] if order == 3 else i2[n]

            if cnt == 1:
                d1[cnt - 1] = s[cnt - 1]
                d2[cnt - 1] = d1[cnt - 1]
                d3[cnt - 1] = d2[cnt - 1]
            else:
                # 一级差分
                v = s[cnt - 1] - s[cnt - 2]
                if v < 0:
                    v += mod
                d1[cnt - 1] = v

                # 二级差分
                v = d1[cnt - 1] - d1[cnt - 2]
                if v < 0:
                    v += mod
                d2[cnt - 1] = v

                # 三级差分
                v = d2[cnt - 1] - d2[cnt - 2]
                if v < 0:
                    v += mod
                d3[cnt - 1] = v

    # 选择输出级（sinc2 -> d2；sinc3 -> d3）
    d_temp = d2 if order == 2 else d3

    # 重定标 + 丢弃前 (order_i+1) 个输出
    d_out = [0] * d_out_len
    write_idx = 0
    for i in range(len2):
        if i < order_i + 1:
            continue

        val = d_temp[i]
        if osr == 32:
            # C 里：d_temp[i] * (order==2 ? 64 : 2)
            val = val * (64 if order == 2 else 2)
        elif osr == 64:
            # C 里：order==2 ? *16 : /4
            val = val * 16 if order == 2 else val // 4
        elif osr == 128:
            # C 里：order==2 ? *4 : /32
            val = val * 4 if order == 2 else val // 32
        elif osr == 256:
            # C 里：order==2 ? *1 : /256
            val = val if order == 2 else val // 256
        elif osr == 512:
            # C 里：order==2 ? /4 : /2048
            val = val // 4 if order == 2 else val // 2048
        else:
            # 贴出的 C 代码对 1024 分支未完整；避免猜测，这里直接报错
            raise ValueError("当前实现仅支持 OSR = 32/64/128/256/512（与给定 C 代码一致）")

        d_out[write_idx] = int(val)
        write_idx += 1

    # print(d1)
    # print(d2)
    # print(d3)
    return d_out, d_out_len


In [ ]:
data_in = [1 for i in range(1000)] 
# print(data_in)
data_out, length = sinc_filter_order_new_py(data_in, osr=64, order_i=0)
# print(data_out)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 